In [ ]:
import csv
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from numpy import nan
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.utils import resample
plt.style.use('classic')
%matplotlib inline
pd.set_option('display.max_rows', None)

In [ ]:
df_train1 = pd.read_csv('sample_data/X_trainData_1.csv')
df_train2 = pd.read_csv('sample_data/Y_trainData_1.csv')
df_test = pd.read_csv('sample_data/X_testData_1.csv')
df_test = df_test.loc[:, df_test.columns != 'ID']

In [ ]:
#temp = df_train1.isin([0]).sum(axis=0)
#temp

In [ ]:
#output = df_train2.drop_duplicates()
#output.groupby('Unnamed: 0').size()
print(df_train1.shape)
print(df_train2.shape)
print(df_test.shape)

(3393, 222)
(3393, 11)
(1000, 222)


In [ ]:
res = pd.concat([df_train1, df_train2], axis=1)
res = res[res.AMK!= -1]
# res2= res[res.AMK==1]
# res3=res[res.AMK==0]
# df_upsampled = resample(res3,replace=True, n_samples=227,random_state=1)
res=pd.concat([res2, df_upsampled], axis=0)
y = res['AMK']
X = res.drop(['AMK'], axis = 1) 
X = X.drop(['INH'], axis = 1) 
X = X.drop(['RIF'], axis = 1) 
X = X.drop(['CIP'], axis = 1) 
X = X.drop(['PZA'], axis = 1) 
X = X.drop(['EMB'], axis = 1)
X = X.drop(['CAP'], axis = 1)
X = X.drop(['STR'], axis = 1)
X = X.drop(['OFLX'], axis = 1)
X = X.drop(['MOXI'], axis = 1)  
X = X.drop(['KAN'], axis = 1)    
print(y.shape)
print(X.shape)
# print(res2.shape)
# print(res.shape)
# print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.1)

(1354,)
(1354, 222)


In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
  
defining parameter range 
param_grid = {'C': [0.01,0.1,0.2,1,2,10,4],  
              'gamma': [0.1,0.01,1,0.2,0.02,0.3,0.03,2], 
              'kernel': ['rbf']}  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 2,scoring='roc_auc') 
# # fitting the model for grid search 
grid.fit(X_train, y_train) 
print(grid.best_params_) #Use these to train SVM model
model = SVC(C=10, kernel='rbf', degree=3, gamma=0.2,probability=True)
model.fit(X_train, y_train)
y_pred = model.predict_proba(df_test)
df = pd.read_csv('sample_data/Y_testData_1_nolabels_AMK.csv')
ID = df['ID'].tolist()
# print(y_pred.shape)
# print(type(y_pred))
# print(len(y_pred))

In [ ]:
# Using the random grid cv to search for best hyperparameters
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 50)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 200, num = 15)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [1, 2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# Creating the base model to tune
rf = RandomForestRegressor()
# Using 3 fold cross validation, 
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 4, verbose=2, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
print(rf_random.best_params_) #Use these to train the next model
model = RandomForestClassifier(n_estimators=1008, criterion='gini', max_depth=23, max_features='sqrt',min_samples_split=2, min_samples_leaf=1,bootstrap=False)
model.fit(X_train, y_train)
print(best_params)
y_pred = model.predict_proba(df_test)
df = pd.read_csv('sample_data/Y_testData_1_nolabels_AMK.csv')
ID = df['ID'].tolist()
print(y_pred.shape)

{'learning_rate': 0.3, 'max_delta_step': 1, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.9}
(1000, 2)


In [ ]:
model = xgb.XGBClassifier(learning_rate=0.27,gamma=0.06,max_depth=5,min_child_weight = 1, eval_metric='auc', subsample=0.98,scale_pos_weight=1.47,objective='binary:logistic')
cv_params = {'max_depth': [3,4,5,6,10], 'min_child_weight': [1,2,3,4], 'subsample': [0.8,0.9,1], 'max_delta_step': [0,1,2,4], 'learning_rate':  [0.1, 0.15, 0.2, 0.25, 0.3]}  
gscv = GridSearchCV(xgb.XGBClassifier(), cv_params, scoring = 'roc_auc', cv = 5)#running grid search cv on model 
gscv.fit(X_train, y_train)#fitting model
# gscv.grid_scores_#displaying scores
best_params= gscv.best_params_#Use these to train the XGB model
model = xgb.XGBClassifier(learning_rate=0.3,max_depth=6,min_child_weight =1, eval_metric='auc',max_delta_step=1)
model.fit(X_train, y_train)
y_pred = model.predict_proba(df_test)
df = pd.read_csv('sample_data/Y_testData_1_nolabels_AMK.csv')
ID = df['ID'].tolist()
print(y_pred.shape)

(1000, 2)


In [ ]:
li_id = []
li_val = []
for i in ID:
    # print(i, y_pred[i-1])
    li_id.append(i)
    li_val.append((y_pred[i-1][1]))
csv = pd.DataFrame({'ID': li_id, 'AMK' :li_val})
csv.to_csv('AMK-submit.csv', index=False, header=True) 

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred)) 
print(roc_auc_score(y_test, y_pred))  

[[ 18   3]
 [  0 115]]
              precision    recall  f1-score   support

           0       1.00      0.86      0.92        21
           1       0.97      1.00      0.99       115

    accuracy                           0.98       136
   macro avg       0.99      0.93      0.96       136
weighted avg       0.98      0.98      0.98       136

0.9285714285714286
